# import packages

In [1]:
# !pip install transformers[torch]==4.3

In [2]:
import os
import torch
import pandas as pd
from transformers import BertModel, DistilBertModel
from tqdm import tqdm
# import kobert_tokenizer

# bring pre-trained kobert model

In [4]:
# pretrained_bertmodel = BertModel.from_pretrained("monologg/kobert")
# 참고
# 아레 코드랑 결과 같음
# model_skt = BertModel.from_pretrained("skt/kobert-base-v1")

In [3]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

# data sets

In [4]:
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name = "merged_data_set_AIR.xlsx"
data_set = pd.read_excel(os.path.join(data_path, file_name))
data_set.shape

(48000, 10)

In [5]:
data_set.columns = ['no',
                    'uid',
                   'publisher',
                   'title',
                   'summary',
                   'content',
                   'content_url',
                    'update_at',
                   'importance',
                   'polarity']
data_set = data_set.reset_index()

In [6]:
data_set.head(1)

,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


# Tokenizer

In [7]:
from kobert_transformers import get_tokenizer
tokenizer = get_tokenizer()

In [8]:
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

## For title

In [9]:
data_set_for_title = data_set[['uid', 'title']]
data_set_for_title["token_len"] = ""
data_set_for_title["tokenization"] = ""
# data_set_for_title.reset_index()
data_set_for_title = data_set_for_title.reset_index()

<ipython-input-9-ebfe296a4add>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["token_len"] = ""
<ipython-input-9-ebfe296a4add>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["tokenization"] = ""


In [72]:
data_set_for_title.head(1)

,index,uid,title,token_len,tokenization
0,0,354652739944452421,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',,


In [73]:
data_set_for_title.tail(1)

,index,uid,title,token_len,tokenization
47999,47999,659762898834428055,"""최신 갤럭시·아이폰, 4명 중 1명이 빌려쓴다""",,


In [74]:
for i in tqdm(range(data_set_for_title.shape[0])):
    tokenized = tokenizer.tokenize(str(data_set_for_title['title'][i]))
    data_set_for_title["token_len"][i] = len(tokenized)
    data_set_for_title["tokenization"][i] = ",".join(tokenized)

  0%|                                                                                        | 0/48000 [00:00<?, ?it/s]<ipython-input-74-9291174cddaf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["token_len"][i] = len(tokenized)
<ipython-input-74-9291174cddaf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["tokenization"][i] = ",".join(tokenized)
100%|██████████████████████████████████████████████████████████████████████████| 48000/48000 [00:26<00:00, 1833.64it/s]


In [75]:
data_set_for_title.head(2)

,index,uid,title,token_len,tokenization
0,0,354652739944452421,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',22,"▁김진,석,▁CJ,헬,로,비,전,▁대표,▁',주,주,총회,▁70%,이상,▁찬성,으로..."
1,1,848994285528420888,[포토] LG유플러스 AR글라스 국내 최초 출시,13,"▁[,포토,],▁LG,유플러스,▁,AR,글,라,스,▁국내,▁최초,▁출시"


In [76]:
data_set_for_title.tail(2)

,index,uid,title,token_len,tokenization
47998,47998,885647782860427818,귀한 몸 된 갤럭신S20 울트라…유통망 불만 높아진다,20,"▁귀,한,▁몸,▁된,▁,갤,럭,신,S,20,▁울,트,라,...,유통,망,▁불,만,▁..."
47999,47999,659762898834428055,"""최신 갤럭시·아이폰, 4명 중 1명이 빌려쓴다""",17,"▁"",최,신,▁갤럭시,·,아이,폰,,,▁4,명,▁중,▁1,명이,▁빌려,쓴,다,"""


In [67]:
import os
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

In [77]:
save_path = "C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\2. data sets\\2. processed data"
data_set_for_title.to_excel(save_path + "/tokenization_for_title.xlsx")

## For summary

In [78]:
data_set.head(1)

,index,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [81]:
data_set_for_summary = data_set[['uid', 'summary']]
data_set_for_summary["token_len"] = ""
data_set_for_summary["tokenization"] = ""
# data_set_for_title.reset_index()
data_set_for_summary = data_set_for_summary.reset_index()

<ipython-input-81-eda7f4fac42b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["token_len"] = ""
<ipython-input-81-eda7f4fac42b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["tokenization"] = ""


In [82]:
data_set_for_summary.head(1)

,index,uid,summary,token_len,tokenization
0,0,354652739944452421,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,,


In [83]:
data_set_for_summary.tail(1)

,index,uid,summary,token_len,tokenization
47999,47999,659762898834428055,SK텔레콤 홍보 모델들이 스마트폰 렌탈 서비스 'T렌탈'을 이용하고 있는 모습.\n...,,


In [91]:
for i in tqdm(range(data_set_for_summary.shape[0])):
    tokenized = tokenizer.tokenize(str(data_set_for_summary['summary'][i]))
    data_set_for_summary["token_len"][i] = len(tokenized)
    data_set_for_summary["tokenization"][i] = ",".join(tokenized)

  0%|                                                                                        | 0/48000 [00:00<?, ?it/s]<ipython-input-91-d3807f5634da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["token_len"][i] = len(tokenized)
<ipython-input-91-d3807f5634da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["tokenization"][i] = ",".join(tokenized)
100%|██████████████████████████████████████████████████████████████████████████| 48000/48000 [00:31<00:00, 1525.82it/s]


In [92]:
data_set_for_summary.head(1)

,index,uid,summary,token_len,tokenization
0,0,354652739944452421,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,90,"▁임,세,영,▁,기자,▁=,▁김진,석,▁CJ,헬,로,비,전,▁대표,가,▁26,일,▁..."


In [93]:
data_set_for_summary.tail(1)

,index,uid,summary,token_len,tokenization
47999,47999,659762898834428055,SK텔레콤 홍보 모델들이 스마트폰 렌탈 서비스 'T렌탈'을 이용하고 있는 모습.\n...,62,"▁SK,텔레콤,▁홍보,▁모델,들이,▁스마트폰,▁,렌,탈,▁서비스,▁',T,렌,탈,'..."


In [94]:
save_path = "C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\2. data sets\\2. processed data"
data_set_for_summary.to_excel(save_path + "/tokenization_for_summary.xlsx")

## For content

In [95]:
data_set_tokened = data_set
data_set_tokened["content_len"] = data_set_tokened['content'].str.len()
data_set_tokened["content_token_len"] = ""
data_set_tokened["content_tokenized"] = ""
data_set_tokened = data_set_tokened.reset_index()

In [96]:
data_set_tokened.head(1)

,level_0,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,content_len,content_token_len,content_tokenized
0,0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,,


In [97]:
for i in tqdm(range(data_set_tokened.shape[0])):
    tokenized = tokenizer.tokenize(str(data_set_tokened['content'][i]))
    data_set_tokened["content_token_len"][i] = len(tokenized)
    data_set_tokened["content_tokenized"][i] = ",".join(tokenized)

  0%|                                                                                        | 0/48000 [00:00<?, ?it/s]<ipython-input-97-6259c18d9197>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_tokened["content_token_len"][i] = len(tokenized)
<ipython-input-97-6259c18d9197>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_tokened["content_tokenized"][i] = ",".join(tokenized)
100%|███████████████████████████████████████████████████████████████████████████| 48000/48000 [00:53<00:00, 902.99it/s]


In [98]:
data_set_tokened.head(1)

,level_0,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,content_len,content_token_len,content_tokenized
0,0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,98,"▁임,세,영,▁,기자,▁=,▁김진,석,▁CJ,헬,로,비,전,▁대표,가,▁26,일,▁..."


In [40]:
data_save_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data"
data_set_tokened.to_excel(data_save_path + "/data_set_tokened.xlsx")

# Pre-processing

In [11]:
data_save_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data"
data_set_tokened = pd.read_excel(data_save_path + "/data_set_tokened.xlsx")

In [74]:
data_set_tokened.publisher.value_counts().to_excel(data_save_path + '/publisher_count.xlsx')

In [217]:
test_dt = pd.DataFrame({"a":[1, 2, 3], "b":["abcdefg", 'cdefg', 'efr']})

In [226]:
test_dt.b.str.index(r"[d]")

ValueError: substring not found

In [220]:
test_dt.str.index("f")

AttributeError: 'DataFrame' object has no attribute 'str'

In [184]:
# 공통
data_set_tokened['content_1'] = data_set_tokened['content'] 
# e-mail address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)].*\Z', " ",regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.rstrip('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', " ",regex = True)

In [212]:
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.index('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]')

ValueError: substring not found

In [197]:
data_set_tokened['content_1'] = data_set_tokened['content'] #.str.replace('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', ' ', regex = True)

## 신문사별
### 1 이데일리
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리 기자들의 비밀공간', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[(]출처: [가-힣]+[)]", ' ', regex = True)
#### 마지막 기자 주소 이후 광고 멘트들. [기자주소]-이후 모든 문자열 삭제로 해결하면 좋겠따. # 1006, 1030
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('20~30 세대를 위한 꿀팁! 정보', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("꿀잼 \'웹툰\'이 매일 매일 공짜...", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("선착순 \'5만명\' 한정, KFC 쿠폰이 공짜~", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("2030을 위한 알짜 정보가 가득", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("최신 웹툰이 매일 공짜. \'웹툰\'", ' ', regex = False)


#
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('chosunbiz.com', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경로보뉴스', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('라이온봇기자', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False) #5253
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('Copyright by JTBC and JTBC Content Hub Co. Ltd.', ' ', regex = False) #5253
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('【서울=뉴시스】', ' ', regex = False) #214
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 본 기사는 한국경제TV와 `금융 AI 전문기업 씽크풀`이 실시간으로 작성한 기사입니다', ' ', regex = False) #213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('ⓒ 한국경제TV, 무단 전재 및 재배포 금지', ' ', regex = False) #213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('전자신문과 금융AI전문기업 씽크풀의 증시분석 전문기자 로봇 ET가 쓴 기사입니다', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('저작권자(c) YTN & YTN PLUS', ' ', regex = False) # 176
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(단위: 천원)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재: [가-힣,]+', ' ', regex = True) #5253
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집: [가-힣,]+', ' ', regex = True) #5253
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]{3}기자', " ", regex=True) # 123
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 선임기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('꿀잼가득', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('청춘뉘우스', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[↑]', " 상승 ", regex=True)

# 공통
## e-mail address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9.]+[@][a-zA-Z.]', ' ', regex = True)

## stock fs ticker no
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[(][0-9]{6}[)]', ' ', regex = True)
# yyyy.mm.dd
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[.][0-9]+[.][0-9]+', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+일', " ", regex=True)

## 사진 설명
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가운데)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 왼쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False) # 213 

# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('...', ' ', regex = False)
## 기타 특수문자 처리
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[△▲■◇◆●]', "", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\n', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…》]', " ", regex=True)

In [166]:
data_set_tokened.loc[8, 'content_1']|

'현지 1위 사파리콤  보건부 당국과 GEPP 착공식윤혜정 KT 빅데이터사업지원단장(전무)이  (현지시간) 사파리콤 본사에서 열린  글로벌 감염병 확산 방지 플랫폼’(GEPP) 착공식에서 인사말을 하고 있다. 사파리콤은 케냐 현지 1위 통신사업자다. KT 제공      KT 가 아프리카에서 감염병 방지를 위한 사회공헌을 위해 현지 통신사·기관들과 협력한다.  KT는  (현지시간) 케냐 나이로비에서 현지 1위 통신사업자 사파리콤  케냐 보건부·정보통신부 등과 함께  글로벌 감염병 확산 방지 플랫폼’(GEPP) 착공식을 진행했다고 밝혔다.  GEPP는 세계 주요 통신사업자들이 로밍 기록 등 통신 이용 데이터를 이용해 세계적인 감염병 확산을 사전에 차단하기 위한 플랫폼이다. 각 국가의 관련 당국과 협업해 보건환경을 개선하자는 취지로 KT가 제안한 뒤 여러 국가 통신사업자 등과 협약을 체결중이다.  사파리콤과 진행하는 이번 착공식은 이런 협약이 구체적인 움직임으로 이어진 첫 사례라고 KT 관계자는 설명했다. KT와 사파리콤은 지난해 협약 체결에 이어 올해 상반기 케냐 법률원  정통부 등 정부 기관들과 워크숍을 통해 현지 적용을 위한 법률·기술 자문을 받았다.  케냐에 구축할 플랫폼은 케냐 보건부에서 설정한 국내외 감염병 정보와 감염병 위험 지역을 여행 중인 사라피콤의 고객 정보를 통합해 분석  활용한다. 분석한 정보를 바탕으로 사파리콤은 고객에게 문자메시지(SMS)를 통해 개인차원의 예방과 응대가 가능할 수 있도록 지원하고  정부 당국은 감염병 노출 현황 파악과 향후 예방정책 마련에도 활용할 수 있다.  케냐는 인근 국가인 DR콩고에서 에볼라 바이러스가 유행해 225명이 사망하는 등 지리적으로 감염병 확산에 민감하며  최근 경제 개발 등으로 외국인의 입국이 늘고 있어 적극적인 모습을 보이고 있다.  반기문 유엔글로벌콤팩트 한국협회 명예회장은 착공식 축하 영상 메시지를 통해 “지속가능개발목표(SDGs) 달성을 위해 민관 협력의 실질적인 사례가 이루어졌다”며 이번 프로젝

In [ ]:
data[0].dropna(subset=["content", "summary"], inplace = True)
data[1].dropna(subset=["content", "summary"], inplace = True)
for i in range(2):
  data[i].loc[:, "content"].replace('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…》]', " ", regex=True, inplace=True)
  data[i].loc[:, "content"].replace('\n', "", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…》]', " ", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('\n', "", regex=True, inplace=True)
  data[i].loc[:, "content"].replace('[가-힣]+ 기자', "", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('[가-힣]+ 기자', "", regex=True, inplace=True)
  data[i].loc[:, "content"].replace('[가-힣]+ 선임기자', "", regex=True, inplace=True)
  data[i].loc[:, "summary"].replace('[가-힣]+ 선임기자', "", regex=True, inplace=True)